# Automated image analysis for dynamic contact angle calculation.
## Script description
For the description of an automated image processing procedure to analyze the moving interface and the dynamic contact angle in a serial sequence of image.  
The procedure contains interface detection, contact angle measurement, outlier removal and output. See the repository `README.md` for general syntax information.

| paramaters | description | required |
| :-          | :-          | :-       |
| `thres_value` | Threshold value for contour detection. Should be adjusted according to image quality. | yes |
| `thres_dist` | Threshold distance value for finding the start and end point of interface contour. Should be adjusted according to image quality. | yes |
| `n_pixels` | Number of pixels considered for tangent. | yes |
| `framerate` | Framerate of images. | yes |
| `pixel_size` | Pixel size of image. | yes |
| `resolution` | Resolution of images. | yes |

In [ ]:
import math
import os
import sys
import cv2 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def get_contours(gray_image, thres_value):
    """
    Extract contour of a grayscale image. 

    Args:
    - gray_image: A grayscale image.
    - thres_value (int): Threshold value for extracting contour. Should be adjusted according to image quality.

    Returns:
    - contours (numpy.array): Extracted contour of the image.
    """
    
    _,thresh = cv2.threshold(gray_image, 150, thres_value, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_tangent(image, point, contour, n_pixels):
    """
    Get tangent of a contour. 

    Args:
    - point (numpy.array): Start point of contour.
    - contour (numpy.array): Contour.
    - n_pixels (int): Number of the closest pixels considered for tangent.

    Returns:
    - closest_to_point (numpy.ndarray): Points of tangent.
    """
        
    # Calculate the distances from each point on the contour to the start point
    distances_to_point = [np.linalg.norm(np.array(p) - np.array(point)) for p in contour]

    # Find the indices of the defined closest points to the start point
    closest_to_point_indices = np.argsort(distances_to_point)[:n_pixels]

    # Extract the closest points to the start point
    closest_to_point = [contour[i] for i in closest_to_point_indices]
    
    # Draw tangent for double-checking
    #cv2.line(image, closest_to_point[0], closest_to_point[-1], (0, 0, 255), 2)

    return closest_to_point

def cal_slope(startPoint, endPoint):
    """
    Calculate the slope of a line using start point and end point. 

    Args:
    - startPoint (numpy.ndarray): Start point of the line.
    - endPoint (numpy.ndarray): End point of the line.

    Returns:
    - slope (float): Slope of the line.
    """
    
    slope = (endPoint[1] - startPoint[1]) / (endPoint[0] - startPoint[0])
    
    return slope
    
def cal_angle(line1, line2):
    """
    Calculate angle between two lines in degree. 

    Args:
    - line1 (list): The first line.
    - line2 (list): The second line.

    Returns:
    - angle_degrees (float): Angle between two lines in degree.
    """
    
    # Calculate the direction vectors of the two lines
    direction_vector_line1 = line1[-1] - line1[0]
    direction_vector_line2 = line2[-1] - line2[0]

    # Calculate the dot product of the direction vectors
    dot_product = np.dot(direction_vector_line1, direction_vector_line2)

    # Calculate the magnitudes of the direction vectors
    magnitude1 = np.linalg.norm(direction_vector_line1)
    magnitude2 = np.linalg.norm(direction_vector_line2)

    # Calculate the cosine of the angle between the lines
    cosine_theta = dot_product / (magnitude1 * magnitude2)

    # Calculate the angle in radians
    angle_radians = np.arccos(cosine_theta)

    # Convert the angle to degrees
    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees


def cal_contact_angle(image, point1, point2, interface, edge1, edge2, n_pixels):
    """
    Calculate the contact angles between interface and two tangents, respectively.
    
    Args:
    - point1 (numpy.ndarray): Start point of interface contour.
    - point2 (numpy.ndarray): End point of interface contour.
    - interface (numpy.ndarray): Interface contour.
    - edge1 (numpy.ndarray): Contour of top wall.
    - edge2 (numpy.ndarray): Contour of bottom wall.
    - n_pixels (int): Number of pixels considered for tangent.
    
    Return:
    - theta1 (float): Contact angle betweem interface contour and top wall in degree.
    - theta2 (float): Contact angle betweem interface contour and bottom wall in degree.
    """

    tangent1 = get_tangent(image, point1, interface, n_pixels)
    tangent2 = get_tangent(image, point2, interface, n_pixels)

    theta1 = cal_angle(edge1, tangent1)
    theta2 = cal_angle(edge2, tangent2)
        
    return theta1, theta2

def detect_interface(path_image1, path_image2, top_margin, thres_value, thres_dist, n_pixels):
    """
    Detect the moving interface front of an image based on the difference between two successive images. 

    Args:
    - path_image1 (str): Path of the first image.
    - path_image2 (str): Path of the second image.
    - top_margin (int): Pixel distance to the top boundary of the original image 
                        to crop the image information on the top left. 
    - thres_value (int): Threshold value for getting contour. Should be adjusted according to image quality.
    - thres_dist (float): Threshold distance value for finding the start and end point of interface contour.
                          Should be adjusted according to image quality.
    - n_pixels (int): Number of pixels considered for tangent.

    Returns:
    - interface (numpy.ndarray): Detected interface contour of the first image.
    - a1 (numpy.ndarray): The first boundary point of the interface contour.
    - b1 (numpy.ndarray): The second boundary point of the interface contour.
    - aa1 (list): N closest pixels from a1 in the wall edge contour.
    - bb1 (list): N closest pixels from b1 in the wall edge contour.
    """
    # Load images
    image1 = cv2.imread(path_image1)
    image2 = cv2.imread(path_image2)
    
    # Cropped images without image information on the top left
    image1 = image1[top_margin:image1.shape[0], :]
    image2 = image2[top_margin:image2.shape[0], :]
    
    # Convert to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Find complete contours
    contours1 = get_contours(gray1, thres_value)
    contours2 = get_contours(gray2, thres_value)
    
    # Find the contour with maximum area
    max_contour1 = max(contours1, key=cv2.contourArea)
    max_contour2 = max(contours2, key=cv2.contourArea)

    # Draw contour with maximum area on images
    #cv2.drawContours(image1, [max_contour1], -1, (0,0, 255), 1)
    #cv2.drawContours(image2, [max_contour2], -1, (0,255, 0), 1)

    """
    Find the contour difference between image1 and image2.
    """

    # Calculate the difference between two images
    diff = cv2.absdiff(gray1, gray2)
    
    # Find the complete contours of the difference
    diff_contours = get_contours(diff, thres_value)

    # Merge contours to one
    merged_contours = np.concatenate(diff_contours)
    
    # Calculate the convex hull of merged contour
    hull = cv2.convexHull(merged_contours)

    # Calculate the moment of the convex hull
    M = cv2.moments(hull)

    # Calculate the centroid
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        centroid = (cx, cy)
        

    """
    Calculate the moving direction of interface:
    line between the centroid of contour difference (M) and closest point (P) on contour1.
    """
    
    min_distance = 9999
    closest_point = None
    for point in max_contour1.squeeze():
        distance = np.linalg.norm(np.array(centroid) - point)
        if min_distance > distance:
            min_distance = distance
            closest_point = point
    
    """
    From the closest point (P) to iterate contour1 and contour2, 
    the first two overlapping points (distance < thres_dist) 
    of two contours are start and end point of interface contour of image1.
    """

    # Remove one dimension of array
    max_contour1 = max_contour1.squeeze()
    max_contour2 = max_contour2.squeeze()
    
    # Find the same closest point on the max_contour1
    unique, counts = np.unique(np.where(max_contour1 == closest_point)[0], return_counts=True)
    duplicates = unique[counts == 2]
    p_idx1 = duplicates[0]

    p1 = p_idx1
    t = True
    p_idxa = None # index
    a1 = None # the first boundary point of interface contour of image1
    while t:
        p1 = p1 - 1
        if p1 < 0:
            p1 = len(max_contour1)-1
        p = max_contour1[p1]
        for idx,h in enumerate(max_contour2):
            dist = np.linalg.norm(np.array(p) - h)
            if dist < thres_dist:
                p_idxa = p1
                a1 = p
                t = False
                break
    
    aa1 = [] # The closest N points from a1
    p1 = p_idxa
    for i in range(n_pixels):
        p1 = p1 - 1
        if p1 < 0:
            p1 = len(max_contour1)-1
        aa1.append(max_contour1[p1])

    p2 = p_idx1
    t = True
    p_idxb = None # index
    b1 = None # the second boundary point of interface contour of image1
    while t:
        p2 = p2 + 1
        if p2 > len(max_contour1)-1:
            p2 = 0
        p = max_contour1[p2]
        for idx,h in enumerate(max_contour2):
            dist = np.linalg.norm(np.array(p) - h)
            if dist < thres_dist:
                p_idxb = p2
                b1 = p
                t = False
                break

    bb1 = [] # The closest N points from b1
    p2 = p_idxb
    for i in range(n_pixels):
        
        p2 = p2 + 1
        if p2 > len(max_contour1)-1:
            p2 = 0
        bb1.append(max_contour1[p2])
        
    #cv2.circle(image1, a1, 3, (0, 0, 255), -1)
    #cv2.circle(image1, b1, 3, (0, 0, 255), -1)
    #cv2.circle(image2, minp, 3, (0, 0, 255), -1)

    # Get the complete interface contour between a1 and b1
    tmp = None
    if p_idxa>p_idxb:
        tmp = p_idxb
        p_idxb = p_idxa
        p_idxa = tmp
        max_contour1 = np.concatenate((max_contour1[0:p_idxa],max_contour1[p_idxb+1:max_contour1.shape[0]+1]), axis=0)
    else:
        max_contour1 = max_contour1[p_idxa+1:p_idxb]

    # Rerrange the pixels of interface contour according to distance
    interface = [a1]
    last = a1
    hul = max_contour1.squeeze().copy()
    for idx,i in enumerate(max_contour1.squeeze()):
        mind = 9999
        minp = None
        for h in hul:
            dist = np.linalg.norm(np.array(last) - h)
            if mind > dist:
                mind = dist
                minp = h
        # print(hul,minp,np.where(hul==minp)[0][0])
        hul = np.delete(hul, np.where(hul==minp)[0][0], axis=0)
        interface.append(minp)
        last = minp
        # if idx==hull.squeeze().shape[0]+1:break
    interface.append(b1)

    return interface, a1, b1, aa1, bb1

def cal_interface_movement(df, pixel_size, framerate, resolution):
    """
    Calculate the interface movement with time using pixel size.
    
    Args:
    - df: Dataframe from detect_interface: image_name, x_top, y_top, x_bottom, y_bottom.
    - pixel_size (float): Pixel size from image.
    - framerate (int): Framerate for time calculation.
    - resolution (int): Resolution of images.
    
    Return:
    - df: New dataframe: image_name, x in m, y in m, time in s, displacement in m, velocity in m/s.
    """
    # Time step size
    dt = 1/framerate
    
    for i in range(len(df.Image)):
        
        df.loc[i, 'time'] = (i + 1) * dt
        df['X1'].values[i] *= pixel_size
        df['X2'].values[i] *= pixel_size
        df.loc[i, 'X_mean'] = (df['X1'].values[i] + df['X2'].values[i]) / 2
        
        # Flip the y-coordinates of pixels
        df['Y1'].values[i] = (resolution - df['Y1'].values[i]) * pixel_size
        df['Y2'].values[i] = (resolution - df['Y2'].values[i]) * pixel_size
        df.loc[i, 'Y_mean'] = (df['Y1'].values[i] + df['Y2'].values[i]) / 2
        
        # Calculate the displacement of the upper and lower point and the mean value
        df.loc[i, 'displacement1'] = (df['X1'].values[i]**2 + df['Y1'].values[i]**2) ** (1/2)
        df.loc[i, 'displacement2'] = (df['X2'].values[i]**2 + df['Y2'].values[i]**2) ** (1/2)
        df.loc[i, 'displacement'] = (df['displacement1'].values[i] + df['displacement2'].values[i]) / 2
        
        # Calculate the interface velocity: double-check defined flow rate
        #df.loc[i, 'velocity_top'] = (df['displacement_top'].values[i] - df['displacement_top'].values[i-1])/dt
        #df.loc[i, 'velocity_bottom'] = (df['displacement_bottom'].values[i] - df['displacement_bottom'].values[i-1])/dt
        #df.loc[i, 'velocity'] = (df['velocity_top'].values[i] + df['velocity_bottom'].values[i]) / 2
    
    return df

def data_visualization(df, pixel_size, framerate, resolution, model):
    """
    Visualize the temporal evolution of data using different models and save figure.
    
    Args:
    - df: Dataframe from detect_interface: image_name, x_top, y_top, x_bottom, y_bottom.
    - pixel_size (float): Pixel size from image.
    - framerate (int): Framerate for time calculation.
    - resolution (int): Resolution of images.
    - model (str): Model used for data visualization. 
                    (displacement, displacement-x, velocity, contact angle, table)
    """
    
    # Plot only interface displacement: relevant for curved channels
    if model == 'displacement':
        plt.plot(df.time,df.displacement1*1000, marker='x', color='#1f77b4', label='outside')
        plt.plot(df.time,df.displacement2*1000, marker='+', color='#ff7f0e', label='inside')
        plt.plot(df.time,df.displacement*1000, marker='o', color='#2ca02c', label='mean')
        plt.xlabel('t [s]', fontsize=12)
        plt.ylabel('interface displacement [mm]', fontsize=12)
        plt.grid()
        plt.legend()
        
    # Plot only interface displacement in x direction: only relevant for straight channel
    elif model == 'displacement-x':
        plt.plot(df.time,df.X1*1000, marker='x', color='#1f77b4', label='top')
        plt.plot(df.time,df.X2*1000, marker='+', color='#ff7f0e', label='bottom')
        plt.plot(df.time,df.X_mean*1000, marker='o', color='#2ca02c', label='mean')
        plt.xlabel('t [s]', fontsize=12)
        plt.ylabel('interface displacement [mm]', fontsize=12)
        plt.grid()
        plt.legend(loc=2)
        
    # Plot only interface velocity
    elif model == 'velocity':
        plt.plot(df.time,df.velocity_top*1000, marker='x', color='#1f77b4', label='top')
        plt.plot(df.time,df.velocity_bottom*1000, marker='+', color='#ff7f0e', label='bottom')
        plt.plot(df.time,df.velocity*1000, marker='o', color='#2ca02c', label='mean')
        plt.xlabel('t [s]', fontsize=12)
        plt.ylabel('interface velocity [mm/s]', fontsize=12)
        plt.grid()
        plt.legend(loc=2)
    
    # Plot only dynamic contact angle
    elif model == 'contact angle':
        plt.plot(df.time,df.theta1, marker='x', color='#1f77b4', label='inside')
        plt.plot(df.time,df.theta2, marker='+', color='#ff7f0e', label='outside')
        plt.plot(df.time,(df.theta2 + df.theta1)/2, marker='o', color='#2ca02c', label='mean')
        plt.xlabel('t [s]', fontsize=12)
        plt.ylabel('dynamic contact angle [degree]', fontsize=12)
        plt.grid()
        plt.legend(loc=2)

    else:
        print("Error: unknown model: " + model + ". Exiting.")
        
    plt.savefig(os.path.join(path_results, model + '_' + data_file + '.png'), bbox_inches='tight', dpi=200, figsize=[15,15])

def write_text(image, text, position):
    """
    Write text on the image with defined position.

    Args:
    - image : Orinigal image.
    - text (str): The text need to be written on image.
    - position (numpy.array):  The position of text on image.

    Returns:
    - image: Image with text.
    """
    
    # Define the font and size
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)
    font_thickness = 2

    # Put the text on the image
    cv2.putText(image, text, position, font, font_scale, font_color, font_thickness)
    
    return image

def cal_point_distance(p1, p2, pixel_size):
    """
    Calculate distance between two points. 

    Args:
    - p1 (numpy.array): The first point.
    - p2 (numpy.array): The second point.
    - pixel_size (float):  Pixel size from image.

    Returns:
    - d (float): Distance between two points.
    """
    
    dx = abs(p1[0] - p2[0])
    dy = abs(p1[1] - p2[1])
    d = (dx**2 + dy**2) ** (1/2) * pixel_size
    
    return d

def remove_outlier(df, col_name, thres_max, thres_min):
    """
    Remove outliers using criterions with range. 

    Args:
    - df: Dataframe to be used for outlier removal.
    - col_name (str): Column name of the dataframe to be used as criterion.
    - thres_max (float): Maximum threshold for outlier removal.
    - thres_min (float): Minimum threshold for outlier removal.    

    Returns:
    - df_clean: Dataframe after outlier removal.
    """
    
    df_remove_max = df[df[col_name] < thres_max]
    df_remove_min = df_remove_max[df_remove_max[col_name] > thres_min]
    df_clean = df_remove_min
    
    return df_clean

In [ ]:
# define path
path_fluid = "Images/02_50%Glycerin/"
path_geometry = "01_straight/"
path_flowrate = "0.0011ml_s_100fps/"

path_images = os.path.join(path_fluid, path_geometry, path_flowrate)
path_output = os.path.join(path_fluid, path_geometry, path_flowrate, 'postprocessed')
path_results = os.path.join(path_fluid, path_geometry, path_flowrate, 'results')

if not os.path.isdir(path_output):
        os.mkdir(path_output)
        
if not os.path.isdir(path_results):
        os.mkdir(path_results)
        
image_files = [file for file in os.listdir(path_images) if file.endswith('.jpg')]
image_files.sort()

# Crop the information on the top left of the image
top_margin = 50

# Optical parameters
pixel_size = 6.566e-6
framerate = 100
resolution = 1024

# Parameters for interface detection
thres_value = 160
thres_dist = 1.2
n_pixels = 10

# Initialize an empty dataframe
df_index = pd.DataFrame(columns=['Image', 'X1', 'Y1', 'X2', 'Y2', 'distance', 'theta1', 'theta2'])

# Iterate through the images, processing them in pairs
for i in range(len(image_files) - 1):

    image1_path = os.path.join(path_images, image_files[i])
    image2_path = os.path.join(path_images, image_files[i + 1])
    
    image1 = cv2.imread(image1_path)
    image1 = image1[top_margin:image1.shape[0], :]
    
    # Call the interface detection function
    interface1, a1, b1, aa1, bb1 = detect_interface(image1_path,image2_path, top_margin, 
                                                    thres_value, thres_dist, n_pixels)
    
    # Call the contact angle calculation function
    theta1, theta2 = cal_contact_angle(image1, a1, b1, interface1, aa1, bb1, n_pixels)
    
    # Write contact angle valus on image1
    image1 = write_text(image1, f'Inside: {theta1:.2f}', (50,100))
    image1 = write_text(image1, f'Outside: {theta2:.2f}', (50,50))

    df_index = df_index.append({'Image': image_files[i], 
                                'X1': b1[0], 'Y1': b1[1], 
                                'X2': a1[0], 'Y2': a1[1],
                                'distance': cal_point_distance(a1, b1, pixel_size),
                               'theta1': theta1, 'theta2': theta2}, ignore_index=True)

    # Draw interface contour on image1
    for j in range(len(interface1)-1):
        p1 =  tuple(interface1[j])
        p2 =  tuple(interface1[j+1])
        cv2.line(image1, p1, p2, (0, 255, 0), 2)

    # Draw wall contours on image1
    for j in range(len(aa1)-1):
        p1 =  tuple(aa1[j])
        p2 =  tuple(aa1[j+1])
        cv2.line(image1, p1, p2, (0, 0, 255), 2)
    for j in range(len(bb1)-1):
        p1 =  tuple(bb1[j])
        p2 =  tuple(bb1[j+1])
        cv2.line(image1, p1, p2, (0, 0, 255), 2)

    # Save processed image1 with original name
    processed1 = os.path.splitext(image_files[i])[0] + ".jpg"
    cv2.imwrite(os.path.join(path_output, processed1), image1)

print("All images postprocessed.")

In [ ]:
# Get the raw dataframe for interface movement and dynamic contact angle
df_raw =  cal_interface_movement(df_index, pixel_size, framerate, resolution)

# Optional: Outlier removal with channel width constraint
df_dist = remove_outlier(df_raw, 'distance', 582e-6, 522e-6)

# Optional: Outlier removal with contact angle constraint for dynamic contact angles on both walls
df_ca = remove_outlier(df_dist, 'theta1', 120, 80)
df_final = remove_outlier(df_ca, 'theta2', 120, 80)
#display(df_final)

# Save the data in csv
data_file = 'threshold_' + str(thres_value) + '_distance_' + str(thres_dist)
df_final.to_csv(os.path.join(path_results, data_file + '.csv'), index=False)

In [ ]:
# Plot the dynamic contact angle with time and save the figure
data_visualization(df_final, pixel_size, framerate, resolution, model="contact angle")

In [ ]:
# Plot the interface displacement with time and save the figure: for curved channels
data_visualization(df_final, pixel_size, framerate, resolution, model="displacement")

In [ ]:
# Plot the interface displacement in x-direction with time and save the figure: for straight channel
data_visualization(df_final, pixel_size, framerate, resolution, model="displacement-x")